In [1]:
from dotenv import load_dotenv
from langchain_community.llms import YandexGPT
import asyncio

from langchain_core.globals import set_debug
#set_debug(True)

load_dotenv()

True

In [2]:
llm = YandexGPT(
    temperature=0,
    model_name="yandexgpt"
)

In [3]:
from langchain.chains import LLMMathChain # Цепочка, интерпретирующая промпт и запускающая интерпретатор Python для вычислений
from langchain.agents import Tool

In [4]:
# initialize the math tool

    
     


llm_math = LLMMathChain.from_llm(llm)

async def strip_input(*args, **kwargs):
     print (args)
     print (kwargs)
     global llm_math
     return await llm_math.ainvoke(*args, **kwargs)

math_tool = Tool(
    name='Калькулятор',
    func=llm_math.invoke,
    coroutine=strip_input,
    description='Полезно, когда необходимо решить вопросы с математическими выражениями.'
)

# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]  # is a list of possible Tools that your agent can use

In [5]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10
)

C:\Users\Tempest\AppData\Local\Temp\ipykernel_22256\2579540736.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  zero_shot_agent = initialize_agent(


In [6]:
await zero_shot_agent.ainvoke("Чему равен квадратный корень из 1215?")



> Entering new AgentExecutor chain...
Чтобы найти квадратный корень из числа 1215, нужно воспользоваться калькулятором.
Action: Калькулятор
Action Input: {"expression": "sqrt(1215)"}('{"expression": "sqrt(1215)"}',)
{}


ValueError: unknown format from LLM: ```
from numexpr import evaluate

expression = "sqrt(1215)"
result = evaluate(expression)
print(result)
```
...numexpr.evaluate("sqrt(1215)")...

In [7]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search_tool = Tool.from_function(
    func=search.run,
    name="Поиск",
    description="полезно, когда нужно произвести поиск информации в Интернете"
)

In [11]:
tools = [search_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
    max_iterations=5
)

In [12]:
agent.invoke("""Как выводить текст в консоль python?""")



> Entering new AgentExecutor chain...
Action: Поиск
Action Input: "как выводить текст в консоль python"
Observation: Аргумент file функции print() в Python. Аргумент file позволяет перенаправить вывод данных не в консоль, а в указанный файл. Это полезно, когда нужно записать результаты работы программы в файл, чтобы их потом как-нибудь ... Как по умолчанию задаётся параметр end в функции print(), и что это означает для вывода данных на консоль?. по умолчанию параметр end равен нулю, что означает отсутствие вывода.. по умолчанию параметр end равен символу \n, что ... Ввод и Вывод Данных в Python: Как Input() и Print() Делают Программу Интерактивной? 15 марта 2025 admin Базовый синтаксис Ввод и вывод данных в Python — это важнейшие аспекты разработки любой программы. Как вывести текст на консоль в Ruby; Как вывести текст на консоль в PHP; 1. Использование функции print() Встроенная функция print() в Python позволяет вывести строку на консоль. Пример использования: Расскажем о том какие

{'input': 'Как выводить текст в консоль python?',
 'output': 'Agent stopped due to iteration limit or time limit.'}